- author: Lee Meng
- date: 2019-09-21 17:22
- title: 神經網路的本質：直觀理解神經網路 & 線性代數
- slug: the-essence-of-neural-network-better-understanding-of-nn-and-linear-algebra
- tags: Manim, Python
- summary: s
- description: s 
- image: TwoLayersReLUInBetweenSolveHardTwoCurves.jpg
- status: draft

!quote
- 這是篇幫助你直觀理解神經網路的科普文。讀完本文，你將能夠深刻地體會神經網路與線性代數之間的緊密關係，奠定深度學習之旅的基礎。

（小提醒：因本文圖片與動畫皆為黑色背景，強烈推薦用左下按鈕以 Dark Mode 閱讀本文）

這是個眾人對人工智慧（**A**rtificial **I**ntelligence, AI）趨之若鶩的時代。此領域近年的蓬勃發展很大一部份得歸功於[深度學習](https://leemeng.tw/deep-learning-resources.html)以及[神經網路](https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)的研究。現行的深度學習框架（framework）也日漸成熟，讓任何人都可以使用 [TensorFlow](https://www.tensorflow.org/overview) 或 [PyTorch](https://pytorch.org/) 輕鬆建立神經網路，解決各式各樣的問題。

舉例而言，你在 30 秒內就可訓練出一個能夠辨識數字的神經網路：

```python
# 此例使用 TensorFlow，但各大深度學習框架的實現邏輯基本上類似
import tensorflow as tf

# 載入深度學習 Hello World: MNIST 數字 dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 建立一個約有 10 萬個參數的「小型」神經網路
# 在現在模型參數動輒上千萬、上億的年代，此神經網路不算大
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# 選擇損失函數、optimizer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# 訓練模型
model.fit(x_train, y_train, epochs=5)

# 訓練後的 NN 在測試集上可得到近 98% 正確辨識率
model.evaluate(x_test, y_test)

# 實際測試結果
# loss: 0.0750 - accuracy: 0.9763
```

是的，扣除註解不到 15 行就可以把讀取數據、訓練 `model` 以及推論全部搞定了。這邊秀出程式碼只是要讓你感受一下透過框架建立一個神經網路有多麽地「簡單」。事實上，這也是絕大多數線上課程以及教學文章**能**教你的東西。對此數字辨識應用有興趣的讀者稍後也可自行參考[ TensorFlow 的 Colab 筆記本](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb)。

我等等要秀給你看的任何一個神經網路都要比 `model` 還簡單個一萬倍，但了解並觀察這些神經網路的運作將成為你的 AI 旅程中最有趣的經驗之一。

## 一些會有幫助的背景知識

本文將透過不少動畫帶你理解神經網路（**N**eural **N**etwork, 後簡稱為 NN）與[線性代數（Linear Algebra）](https://www.youtube.com/watch?v=fNk_zzaMoSs)之間的緊密關係。不過具備些背景知識將能讓你有更深的體會：
1. 能夠讀懂文章開頭建立 NN 的 [Python](https://www.python.org/) 程式碼
2. 了解線上課程都會教的[超基本 NN 概念](https://www.youtube.com/watch?v=Dr-WRlEFefw)
    - 何謂[全連接層（Fully Connected Layer）](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E5%85%A8%E9%80%A3%E6%8E%A5%E5%B1%A4)
    - 何謂參數以及如何估計全連接層的參數量
    - 常見的 activation functions 如 [ReLU](https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E6%95%B4%E6%B5%81%E5%87%BD%E6%95%B0)
3. 基本的線性代數概念如矩陣相乘、向量空間

別擔心，這些是 nice-to-have，就算你沒背景知識也能直接繼續閱讀。

!mp4
- images/manim/00010.mp4
- 一個簡單 NN 嘗試解決二元分類的過程（線性轉換）

另外因為篇幅有限，本文會把焦點放在**已經訓練好**的 NN，不會特別說明[訓練神經網路](https://www.youtube.com/watch?v=IHZwWFHWa-w)的細節。

## 深度學習框架操作容易，但你真的了解神經網路嗎？

讓我們再次回到文章開頭透過 [Keras](https://www.tensorflow.org/guide/keras) 建立的神經網路 `model`：

```python
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
```

就跟你在多數深度學習文章裡頭看到的，現在要使用深度學習框架建立**基本的** NN 非常容易，只要當作疊疊樂一個個 layer 疊上去就好了。下圖則將此 `model` 用視覺上更容易理解的方式呈現：

!image
- manim/mnist-simple.jpg
- 輸入是 28*28 = 784 維的圖片像素，輸出則是 10 個數字類別的簡單 2-layers NN

透過 API，我們也可以輕鬆查看整個 NN 的參數量：

In [4]:
#ignore
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


儘管擁有 10 萬個可訓練參數，此 `model` 是個在深度學習領域裡只能被歸類在 Hello World 等級的可憐 NN。畢竟這世界很瘋狂，[我們以前討論過的 BERT](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html) 以及 [GPT-2](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html) 都是現在 NLP 界的知名語言模型，而它們可都是擁有**上億**參數的強大 NN。那可是此 `model` 的 100 倍大。

但先別管 BERT 或 GPT-2 了，就算是這個 Hello World 等級的 NN，你真的覺得你對它的運作機制有足夠的理解嗎？

講白點，儘管現在路上隨便拉個人都會使用 TensorFlow 或是 PyTorch 來建立神經網路，我認為許多人（包含剛入門的我）對最基本的神經網路都沒有足夠**直觀**的理解。

!image
- manim/three-key-components.jpg
- 構成本文的關鍵三要素：矩陣運算、二元分類以及神經網路

為了讓你能夠更直觀地理解神經網路，我將透過二元分類（Binary Classification）任務說明神經網路以及線性代數之間的緊密關係。前言很長，但如果你想要了解神經網路的本質，或是想要為自己之後的 AI 之旅打下良好基礎，那我會建議你繼續往下閱讀：）

## 用二元分類連結神經網路 & 線性代數

[二元分類](https://en.wikipedia.org/wiki/Binary_classification)是[機器學習（Machine Learning）](http://speech.ee.ntu.edu.tw/~tlkagk/courses_ML19.html)領域裡一個十分基本的任務，其目標是把一個集合裡的所有數據點（data points）依照某種分類規則劃分成**兩**個族群（groups）或類別（classes）。比方我們之間看過的[貓狗圖像辨識](https://demo.leemeng.tw/)。不過在這篇文章裡，我假設所有數據點**最多只有 2 個維度（dimensions）**。

如果你讀過之前的文章，可能會覺得這假設是在「羞辱」我們。畢竟我們已用過神經網路來：
- [執行假新聞偵測（BERT）](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html)
- [生成新金庸小說（GPT-2）](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html)
- [把英文翻成中文（Transformer）](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html)
- [生成新海誠動畫（CartoonGAN）](https://leemeng.tw/generate-anime-using-cartoongan-and-tensorflow2.html)

以上這些進階任務跟本文的 **2 維**二元分類相比要複雜許多，自然也十分有趣且具有挑戰性。如果你有興趣深入了解，稍後可以點擊連結來查看相關文章。但這篇之所以用二元分類作為目標任務是因其十分單純，所以我們能夠透過它來一窺神經網路的本質。

以下是一個資料集：

!mp4
- images/manim/ShowLinearSeparableDataPoints.mp4
- 包含兩類別（曲線）的 2 維資料集

在此資料集裡，兩曲線分別來自不同類別，各自包含 100 個數據點 $x$。每個數據點 $x$ 也可以很自然地用 2 維的 $\left (x_{coord}, y_{coord}  \right )$ 座標來表示。你可以從右下角得知黃點的標籤 $y = 0$，藍點為 $1$。

另外，圖中也描繪了此向量空間中的基底向量（basis vector）：
- x 軸上藍色的 $\vec{i}$
- y 軸上紅色的 $\vec{j}$ 

那麼要如何分類這個資料集呢？複習[ AI For Everyone 裡的其中一個重要概念](https://leemeng.tw/10-key-takeaways-from-ai-for-everyone-course.html)：

!quote
- 目前多數的機器學習以及 AI 應用本質上都是讓電腦學會一個映射函數（Mapping Function），幫我們將輸入的數據 x 轉換到理想的輸出 y。
- Andrew Ng

套用相同概念，要處理這個分類任務，我們要問的問題就變成「給定所有藍點與黃點 $x = \left (x_{coord}, y_{coord}  \right )$，我們能不能找出一個函數 $f$，將這些 2 維數據 $x$ 完美地**轉換**到它們對應的 1 維標籤 $y$ 呢？」

換句話說，我們想要找出一個 $x$ 的函數 $f(x)$，使得以下式子成立：

\begin{align}
f(x) 
& = f(\begin{bmatrix} x_{coord} \\ y_{coord} \end{bmatrix}) \\
& = f(\begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix}) \\
& = y
\end{align}


如果我們能找到符合這個條件的 $f(x)$，就能瞬間預測出一個數據點 $x$ 離哪個類別比較近了。當然，我們有無窮多種 model $f(x)$ 的方法。但在線性代數的世界裡，我們可以用矩陣運算的形式定義 $f(x)$：

\begin{align}
f(x) & = W x + b \\
  & = \begin{bmatrix} w_{11} & w_{12}  \end{bmatrix} x + b \\ 
  & = \begin{bmatrix} w_{11} & w_{12}  \end{bmatrix} \begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix} + b \\ 
  & = y
\end{align}

希望你還記得國中、高中或是大學裡任何一位線性代數老師的諄諄教誨。

在上面的式子裡：
- $x$ 是一個 2 維的 column vector
- $W$ 是一個 1 x 2 的[權重矩陣（weight matrix）](https://zh.wikipedia.org/wiki/%E7%9F%A9%E9%98%B5)
- $b$ 為偏差（bias），是一個純量（scalar）

如果我們先暫時忽略 $b$，事實上 $f(x)$ 對輸入 $x$ 做的**轉換**跟多數神經網路都會使用到的[全連接層（**F**ully **C**onnected Layer，後稱 FC）](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E5%85%A8%E9%80%A3%E6%8E%A5%E5%B1%A4)的作用是完全相同的。換句話說，使用一層 FC 的 NN 基本上就是在做矩陣運算（假設激勵函式為線性）。因此這個形式的 $f(x)$ 與神經網路之間有非常美好的對應關係：

!mp4
- images/manim/SymbolicOneByTwoMatrixAndNN.mp4
- 1 x 2 矩陣運算與 1-Layer NN 的對應關係

沒錯，透過矩陣運算，我們建立了這世上最簡單的 1-Layer 神經網路！而之所以是 1-Layer，原因在於 NN 的第一層為原始數據（raw data），第二層才是我們實際新定義的神經網路（FC）。矩陣運算跟 FC 的對應關係家喻戶曉，但這應該是你第一次看到兩者共舞。

從**線性代數**的角度來看，我們是透過權重矩陣 $W$ 對 2 維的 $x$ 進行線性轉換後得到 1 維的 $y$；而以**神經網路**的角度檢視，我們則是將以 2 維向量表示（represent）的數據點 $x$ 透過與權重 $W$ 進行加權總和後得到新的 1 維 representation $y$。這是常有人說神經網路在做[表徵學習（Representation Learning）](https://arxiv.org/abs/1206.5538)的原因。

另外值得一提的是矩陣 $W$ 裡的參數 $w_{mn}$ 實際上對應到 NN 某一層中第 $n$ 個神經元（neuron）連到其下一層中第 $m$ 個神經元的**邊（edge）**。

而就跟文章開頭看到的程式碼一樣，要使用 TensorFlow 定義這個神經網路也十分容易：

In [63]:
import tensorflow as tf
model = tf.keras.models.Sequential()
# 將 2 dim 轉成 1 dim
model.add(tf.keras.layers.Dense(1, 
                                use_bias=False,
                                input_shape=(2, )))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


扣除 bias 以後整個神經網路的確只有 2 個參數。為了讓你加深印象，讓我們將數字實際代入 $W$ 與 $x$：

\begin{align}
W = \begin{bmatrix} 1 & 5  \end{bmatrix} \\ 
x = \begin{bmatrix} 3 \\ 3 \end{bmatrix}
\end{align}

接著再看一次剛剛的運算過程：

!mp4
- images/manim/NumberOneByTwoMatrixAndNN.mp4

這邊我刻意讓 $x_{1} = x_{2}$。你可以清楚地看到不同大小的 $w_{mn}$ 可以讓相同的輸入維度 $x_{n}$ 給與輸出值 $y_{m}$ 不同程度的影響力，這也是 $W$ 之所以被稱之為「權重」的原因。

你現在應該已經了解最基本的矩陣運算還有神經網路的概念，以及兩者之間的緊密關係了。更美妙的是，如果你了解[線性代數的本質](https://youtu.be/fNk_zzaMoSs)，就會知道函數 $f(x) = W x + b = y$ 事實定義了兩個簡單轉換來將 2 維的輸入 $x$ 依序轉換成 1 維輸出 $y$：
- 線性轉換：$W$
- 位移：$b$

你可能也已經看過[線性轉換（linear transformation, or linear map）](https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E6%98%A0%E5%B0%84)的數學定義：

!image
- dark
- manim/formal-linear-transformation-formula.jpg

這是所有線性轉換都具備的[可加性（additivity）](https://en.wikipedia.org/wiki/Additive_map)與[齊次性質（homogeneity）](https://zh.wikipedia.org/wiki/%E9%BD%90%E6%AC%A1%E5%87%BD%E6%95%B0)。不過別擔心，在本文裡你不需了解這些定義也能直觀地理解線性轉換。用比較不嚴謹的說法，線性轉換會將一個向量空間（vector space）**旋轉**或**伸縮**（或是兩者皆做），但不會扭曲該空間。

要直觀瞭解這個概念，讓我們再次將幾個數字代入 $f(x)$ 裡頭：

\begin{align}
f(x) & = \begin{bmatrix} w_{11} & w_{12}  \end{bmatrix} \begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix} + b \\ 
 & = \begin{bmatrix} 1 & 1  \end{bmatrix} \begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix} + (-2) 
\end{align}

這次假設 $w_{11} = w_{12} = 1, b = -2$。另外別忘了我們前面說過的，這個 $f(x)$ 實際上也對應到一個 1-Layer NN。我們可以看看這個神經網路會怎麼轉換 2 維空間裡頭的 $x$：

!mp4
- images/manim/ApplyMatrixAsSingleLinearTransformation.mp4

這個 NN 做的轉換非常地簡單，但是隱含不少前面提過的重要概念。

首先，以**神經網路**的角度而言，此 2 維向量空間裡頭的任一數據點 $ x = \left (x_{coord}, y_{coord}  \right )$ 都對應到左上 NN 第一層（輸入層）中的兩個神經元。權重 $W$ 則以**邊**的方式呈現，負責將第一層 2 神經元的值透過不同比重**送去啟動**（activate）下一層的神經元。啟動下層的神經元以後，NN 只要再將純量 $b$ 加上去即完成轉換。

以**線性代數**的視角來看，矩陣 $W$ 則定義了一個線性轉換，此轉換說明了原向量空間 $V_{original}$ 裡的 2 個基底向量 $\vec{i}$、$\vec{j}$ 在**轉換後**的空間 $V_{transformed}$ 裡頭的位置。我們透過 $W = [1, 1]$ 將 y 軸的 $\vec{j}$ 放到 $V_{transformed}$（跟 x 軸重疊的數線）裡跟 $\vec{i}$ 一樣的位置（$w_{12} = 1$）；而原 x 軸上的 $\vec{i}$ 則保持在原位（$w_{11} = 1$）。

為了讓轉換過後的 $\vec{j}$ 能在指定的位置，y 軸順時針旋轉並壓縮到了 x 軸上。你也能看到原 2 維空間裡頭的每個數據點 $x$ 都跟著 $\vec{j}$ 一起被轉換成 1 維數線（Number line）上的一個值，而不再是 2 維的座標。透過 $W$ 被轉換到一維空間以後，該數線上的每個數據點只要再被加上純量 $b$ 就完成兩步驟的轉換。

這邊的重點是，**神經網路**與**線性代數**雖然看似風馬牛不相及，實際上兩者做的是同件事情：利用 $f(x)$ 將輸入 $x$ 進行一系列幾何轉換後輸出 $y$。到此為止，你應該已能瞭解為何[我曾說過神經網路是個映射函數的理由了](https://demo.leemeng.tw/#%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF%EF%BC%9A%E4%B8%80%E5%80%8B%E6%98%A0%E5%B0%84%E5%87%BD%E6%95%B8)。

## 能夠解決二元分類的神經網路長得是什麼模樣？

我們在上一節看到，一個 1-Layer 神經網路可以透過一個全連接層（線性轉換 $W$ + 位移 $b$）將 2 維輸入 $x$ 轉換到一維輸出 $y$。但你剛剛應該也已經注意到，轉換後的 $y$ 實在不是什麼好的分類結果：

!mp4
- images/manim/ApplyMatrixAsSingleLinearTransformationAndShowAccuracy.mp4

你的時間寶貴，因此我事先將本文的精華都濃縮在底下這個 1 分鐘的短片了。此影片展示了一個簡單神經網路解決二分類任務的**完整過程**。我等等會更仔細地說明影片內容，但現在請你馬上點擊播放鍵觀看吧！

!mp4
- options: no-loop, no-autoplay, controls
- images/manim/TwoLayersReLUInBetweenSolveHardTwoCurves.mp4

順帶一提，這是一個只有 9 個參數的神經網路，其規模跟現在媒體整天在報導的 A.I. 相比可說是滄海一粟。

!quote
- 但這是我看過最美麗、直觀的神經網路運作。

我不知道看影片前的你對神經網路或是線性代數的理解程度，但我相信很多人都能在這短短的一分鐘內（重新）獲得些啟發。如果你願意，我強烈建議你至少再看一次影片並在需要時暫停咀嚼。裡頭有很多十分基本但重要的 NN 概念值得掌握。

你有再看一遍嗎？讓我幫你把影片裡隱含的重要概念一一列出：
- 神經網路是將輸入（維度）做一連串簡單轉換，最後得到理想輸出（維度）的函式
- 最基本且常見的神經網路是「層」為單位，每一層的矩陣相乘事實上都在做線性轉換
- 線性轉換基本上就是對輸入空間的數據做旋轉、縮放、延伸等轉換
- 層跟層之間常會透過非線性轉換函式來提升神經網路整體的轉換能力
- 神經網路也常被視為是在做 representation learning，因為每一層的轉換都將該層輸入的數據轉換成更適合達到任務目標的形式
- 透過對輸入數據做一連串適當的
- 針對某些任務，特定的神經網路架構有其能力極限，我們透過學習從該架構中找出一組參數，讓神經網路做適合的轉換以完成任務


很多東西你可能都知道了，人類是視覺動物，

## 結論
- 下一代
- 我們在 dl resource 說過神經網路是一連串的幾何轉換